# 📘 Enhanced Result Objects with Statistical Analysis> Access R², confidence intervals, and visualization with enhanced CurveFitResult⏱️ **20-25 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Feature Demo**---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/05_feature_demos/result_enhancements_demo.ipynb)


In [ ]:
# @title Install NLSQ (run once in Colab)
import sys

if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("✅ NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

## 🎯 Learning ObjectivesAfter this tutorial, you'll be able to:1. Access statistical properties (R², RMSE, MAE, AIC, BIC)2. Compute confidence and prediction intervals3. Use backward-compatible result objects4. Generate summary reports and visualizations5. Compare multiple models systematically---

## 🔬 Feature Overview**What problem does this solve?**- Manual calculation of goodness-of-fit metrics is tedious- Confidence intervals require complex statistical code- Model comparison lacks standardized metrics- Visualization requires repetitive plotting code**Enhanced CurveFitResult provides:**- Statistical properties: R², RMSE, MAE, AIC, BIC- Confidence/prediction intervals- Built-in visualization with residual plots- Summary reports- Full backward compatibility with tuple unpacking---

## Setup

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [ ]:
from IPython.display import display

In [ ]:
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np

from nlsq import curve_fit

## Example 1: Basic Statistical PropertiesAccess common goodness-of-fit metrics directly.

In [ ]:
def exponential(x, a, b, c):
    return a * jnp.exp(-b * x) + c

# Generate sample data
np.random.seed(42)
x = np.linspace(0, 10, 100)
y_true = 10 * np.exp(-0.5 * x) + 2
y = y_true + np.random.normal(0, 0.5, size=len(x))

# Fit model
result = curve_fit(exponential, x, y, p0=[10, 0.5, 2])

print('Fitted parameters:')
print(f'  a = {result.popt[0]:.4f}')
print(f'  b = {result.popt[1]:.4f}')
print(f'  c = {result.popt[2]:.4f}')

print('\nGoodness of fit:')
print(f'  R² = {result.r_squared:.6f}')
print(f'  Adjusted R² = {result.adj_r_squared:.6f}')
print(f'  RMSE = {result.rmse:.6f}')
print(f'  MAE = {result.mae:.6f}')

print('\nModel selection criteria:')
print(f'  AIC = {result.aic:.2f}')
print(f'  BIC = {result.bic:.2f}')

## Example 2: Backward CompatibilityResult objects support both modern and traditional usage patterns.

In [ ]:
def linear(x, a, b):
    return a * x + b

np.random.seed(42)
x = np.linspace(0, 10, 50)
y = 2 * x + 1 + np.random.normal(0, 0.5, size=len(x))

# Pattern 1: Traditional tuple unpacking (backward compatible)
popt, pcov = curve_fit(linear, x, y, p0=[1, 1])
print('Pattern 1 (tuple unpacking):')
print(f'  popt = {popt}')

# Pattern 2: Enhanced result object
result = curve_fit(linear, x, y, p0=[1, 1])
print('\nPattern 2 (enhanced result):')
print(f'  result.popt = {result.popt}')
print(f'  result.r_squared = {result.r_squared:.6f}')

# Pattern 3: Unpack enhanced result
popt2, pcov2 = result
print('\nPattern 3 (unpack enhanced result):')
print(f'  popt = {popt2}')
print(f'  Same as Pattern 1? {np.allclose(popt, popt2)}')

print('\n✓ All usage patterns work seamlessly!')

## Example 3: Confidence IntervalsQuantify parameter uncertainty with confidence intervals.

In [ ]:
def power_law(x, a, b):
    return a * x**b

np.random.seed(42)
x = np.linspace(1, 10, 50)
y = 2 * x**1.5 + np.random.normal(0, 2, size=len(x))

# Fit model
result = curve_fit(power_law, x, y, p0=[2, 1.5])

# Get confidence intervals
ci_95 = result.confidence_intervals(alpha=0.95)
ci_99 = result.confidence_intervals(alpha=0.99)

print('Fitted parameters with confidence intervals:')
print('\nParameter    Value       95% CI                    99% CI')
print('-' * 70)
for i, (val, ci95, ci99) in enumerate(zip(result.popt, ci_95, ci_99, strict=False)):
    print(f'p{i:<11} {val:>8.4f}    [{ci95[0]:>7.4f}, {ci95[1]:>7.4f}]    [{ci99[0]:>7.4f}, {ci99[1]:>7.4f}]')

## Example 4: Prediction IntervalsEstimate uncertainty for future observations.

In [ ]:
def quadratic(x, a, b, c):
    return a * x**2 + b * x + c

np.random.seed(42)
x = np.linspace(0, 5, 30)
y = 0.5 * x**2 - 2 * x + 1 + np.random.normal(0, 0.3, size=len(x))

# Fit model
result = curve_fit(quadratic, x, y, p0=[1, -2, 1])

# Get prediction intervals at fitted x values
pi = result.prediction_interval()

print('Prediction intervals at first 5 data points:')
print('\n  x       y_data    y_pred    Lower     Upper')
print('  ' + '-'*50)
for i in range(5):
    print(f'  {x[i]:.2f}    {y[i]:>6.3f}    {result.predictions[i]:>6.3f}    {pi[i,0]:>6.3f}    {pi[i,1]:>6.3f}')

# Get prediction intervals at new x values
x_new = np.array([1.5, 3.0, 4.5])
pi_new = result.prediction_interval(x=x_new)

print('\nPrediction intervals at new x values:')
print('\n  x_new    y_pred    Lower     Upper    Width')
print('  ' + '-'*50)
for i, x_val in enumerate(x_new):
    width = pi_new[i,1] - pi_new[i,0]
    y_pred = result.model(x_val, *result.popt)
    print(f'  {x_val:.2f}     {y_pred:>6.3f}    {pi_new[i,0]:>6.3f}    {pi_new[i,1]:>6.3f}    {width:>6.3f}')

## Example 5: Built-in VisualizationGenerate professional plots with one line of code.

In [ ]:
def gaussian(x, a, mu, sigma):
    return a * jnp.exp(-((x - mu)**2) / (2*sigma**2))

np.random.seed(42)
x = np.linspace(-5, 5, 100)
y = 10 * np.exp(-((x - 1)**2) / (2*1.5**2)) + np.random.normal(0, 0.5, size=len(x))

# Fit model
result = curve_fit(gaussian, x, y, p0=[10, 1, 1.5])

# Plot with residuals
result.plot(show_residuals=True)
plt.savefig('curve_fit_result.png', dpi=150, bbox_inches='tight')
print('✓ Plot saved to curve_fit_result.png')
plt.tight_layout()
plt.tight_layout()
plt.show()


## Example 6: Summary ReportGet comprehensive statistical summary.

In [ ]:
def exponential(x, a, b, c):
    return a * jnp.exp(-b * x) + c

np.random.seed(42)
x = np.linspace(0, 10, 100)
y_true = 10 * np.exp(-0.5 * x) + 2
y = y_true + np.random.normal(0, 0.5, size=len(x))

# Fit model
result = curve_fit(exponential, x, y, p0=[10, 0.5, 2])

# Print summary
result.summary()

## Example 7: Model ComparisonUse AIC/BIC to select the best model.

In [ ]:
# Generate exponential decay data
np.random.seed(42)
x = np.linspace(0, 10, 100)
y_true = 10 * np.exp(-0.5 * x) + 2
y = y_true + np.random.normal(0, 0.5, size=len(x))

# Model 1: Exponential (correct)
def exponential(x, a, b, c):
    return a * jnp.exp(-b * x) + c

# Model 2: Linear (wrong)
def linear(x, a, b):
    return a * x + b

# Model 3: Quadratic (overfitted)
def quadratic(x, a, b, c):
    return a * x**2 + b * x + c

# Fit all models
result_exp = curve_fit(exponential, x, y, p0=[10, 0.5, 2])
result_lin = curve_fit(linear, x, y, p0=[-1, 10])
result_quad = curve_fit(quadratic, x, y, p0=[0, -1, 10])

# Compare
print('Model Comparison:')
print(f"\n{'Model':<15} {'Params':<8} {'R²':<10} {'RMSE':<10} {'AIC':<10} {'BIC':<10}")
print('-'*70)

models = [
    ('Exponential', result_exp),
    ('Linear', result_lin),
    ('Quadratic', result_quad)
]

for name, res in models:
    print(f'{name:<15} {len(res.popt):<8} {res.r_squared:<10.6f} {res.rmse:<10.6f} {res.aic:<10.2f} {res.bic:<10.2f}')

best_idx = np.argmin([r.aic for _, r in models])
print(f'\nBest model (lowest AIC): {models[best_idx][0]}')

## Example 8: Accessing ResidualsResiduals and predictions are directly accessible.

In [ ]:
def sine_wave(x, a, f, phi):
    return a * jnp.sin(2 * jnp.pi * f * x + phi)

np.random.seed(42)
x = np.linspace(0, 2, 50)
y = 3 * np.sin(2*np.pi*2*x + 0.5) + np.random.normal(0, 0.3, size=len(x))

# Fit model
result = curve_fit(sine_wave, x, y, p0=[3, 2, 0.5])

# Access residuals
residuals = result.residuals
print('Residuals:')
print(f'  Mean: {np.mean(residuals):.6f} (should be ~0)')
print(f'  Std:  {np.std(residuals):.6f}')
print(f'  Min:  {np.min(residuals):.6f}')
print(f'  Max:  {np.max(residuals):.6f}')

# Access predictions
predictions = result.predictions
print('\nPredictions:')
print(f'  Shape: {predictions.shape}')
print(f'  Correlation with data: {np.corrcoef(predictions, y)[0,1]:.6f}')

# Verify
manual_residuals = y - predictions
print('\nVerification:')
print(f'  Residuals match (data - predictions)? {np.allclose(residuals, manual_residuals)}')

## 💡 Key Insights1. **Enhanced results** provide instant access to statistical metrics2. **Confidence intervals** quantify parameter uncertainty3. **Prediction intervals** estimate uncertainty for new observations4. **Built-in visualization** saves repetitive plotting code5. **Model comparison** with AIC/BIC for systematic selection6. **Fully backward compatible** with existing code---## 📚 Available Properties**Goodness of Fit:**- `r_squared`: Coefficient of determination (R²)- `adj_r_squared`: Adjusted R² (penalizes extra parameters)- `rmse`: Root mean squared error- `mae`: Mean absolute error- `chi_squared`: Chi-squared statistic**Model Selection:**- `aic`: Akaike Information Criterion- `bic`: Bayesian Information Criterion**Data Access:**- `residuals`: Fit residuals (y - ŷ)- `predictions`: Fitted values (ŷ)- `popt`: Optimal parameters- `pcov`: Covariance matrix**Methods:**- `confidence_intervals(alpha)`: Parameter confidence intervals- `prediction_interval(x, alpha)`: Prediction intervals- `plot(show_residuals)`: Visualization- `summary()`: Statistical summary report---## 🎓 Next Steps- Use enhanced results in your fitting workflows- Compare models with AIC/BIC- Report confidence intervals in publications- Create publication-quality plots with `.plot()`- Explore prediction intervals for forecasting---